In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Predictors of mental health illness** <br>
2014년 설문조사. 기술 사업장에서의 정신건강에 대한 태도, 정신건강 장애 빈도를 측정하는 설문<br>

원본 데이터 셋(캐글) : https://www.kaggle.com/datasets/osmi/mental-health-in-tech-survey<br>
참고 코드 : https://www.kaggle.com/code/kairosart/machine-learning-for-mental-health-1<br>

### 01. 라이브러리 불러오기

In [3]:
import pandas as pd  # 데이터 파일 작업
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, binarize #라벨 인코더
from sklearn.model_selection import train_test_split         #데이터 나누기
from sklearn.datasets import make_classification

In [5]:
from scipy import stats
from scipy.stats import randint

In [6]:
%%time
surv = pd.read_csv("../input/mental-health-in-tech-survey/survey.csv")

In [7]:
surv.shape

In [8]:
surv.head()

In [9]:
surv.tail()

In [10]:
surv.info()

In [11]:
surv.columns

### 02. 데이터 불러오기 / 확인하기
* 1259개행과 27개열(column)의 데이터로 구성되어 있다.
  * 'Timestamp' : 타임스탬프 2014-08-27 ~ 2016-02-01
  * 'Age' : 나이
  * 'Gender' : 성별
  * 'Country' : 국가
  * 'state' : 미국 어느 주에 살고 있는지?
  * 'self_employed' : 자영업자인지?
  * 'family_history' : 정신질환 가족력
  * 'treatment' : 정신질환 치료력
  * 'work_interfere' : 정신 건강 문제가 업무에 방해가 되는지?
  * 'no_employees' : 직원 / 동료의 수
  * 'remote_work' : 시간의 50% 이상 원격(사무실 외부)으로 작업합니까?
  * 'tech_company' : 기술 사업장인지?
  * 'benefits' : 정신건강 혜택이 있는지?
  * 'care_options' : 정신건강 관리 옵션이 있는지?
  * 'wellness_program' : 건강 프로그램에 대해 논의한 적이 있습니까?
  * 'seek_help' : 정신건강 문제와 도움을 구할 수 있는 방법에 대해 리소스를 지공하는지?
  * 'anonymity' : 익명성이 보호되는지?
  * 'leave' : 정신건강상의 병가 가능한지?
  * 'mental_health_consequence' : 정신건강 결과를 open 하는 것이 부정적이라 생각하는지?
  * 'phys_health_consequence' : 신체건강 결과를 open하는 것이 부정적이라 생각하는지?
  * 'coworkers' : 동료와 정신건강 문제를 논의할 의향이 있는지?
  * 'supervisor' : 직속상사와 정신건강 문제를 논의할 의향이 있는지?
  * 'mental_health_interview' : 인터뷰에서 정신건강 문제를 제기할지?
  * 'phys_health_interview'; 인터뷰에서 신체건강 문제를 제기할지?
  * 'mental_vs_physical' : 고용주가 정신건강을 신체건강만큼 중요하게 생각하는가?
  * 'obs_consequence' : 직장에서 정신건강문제의 부정적인 결과를 듣거나 관찰한 적이 있는가?
  * 'comments' : 추가 메모나 의견
  
* 결측치
  * 미국 거주 -> 거주하는 주(state) 확인 : 744개의 응답.
  * 자영업자 여부(self_employed) -> 1241개의 응답.
  * 정신건강 문제가 업무에 방해가 되는지(work_interfere) -> 995개의 응답.
  * 기타 추가적인 메모나 의견(comments) -> 164개의 응답.

* Inspiration 를 보니 다음을 제시하고 있다.
  * 정신 질환 빈도나 정신 건강 태도가 지리적 위치에 따라 어떻게 다른지
  * 직장에서 특정 태도를 예측할 수 있는 변수는 무엇인지


### **->데이터 다양하게 확인해보고, 시각화 해보기를 진행하고자 한다.**

In [12]:
surv.describe()

* describe()를 찍어보면 수치형 변수인 Age에 대한 값만 확인할 수 있다.

### 03. 데이터 정제(Data Cleaning), 정규화(Normalizing)
* 모델을 훈련할 수 있는 데이터를 확인하기 위해서 결측치(NaN), 이상치(outlier)를 확인하는 것.
* 결측값 isnull().sum()
* 평균값으로 NaN 대체 fillna(mean())
* NaN 제거 dropna()
* 데이터를 정제한 후 데이터 범위의 숫자 열 값을 변경하여 값 범위의 차이를 수정하지 않고 공통 척도를 적용하는 것.

### 03-01) 결측치 확인

* isnull() : 결측치 확인
* isnull.sum() : 열 단위
* isnull.sum(axis=1) : 행 단위
* solt_values() : 정렬 오름차순
* solt_values(ascending=False) : 정렬 내림차순

In [13]:
total = surv.isnull().sum().sort_values(ascending=False)
percent = (surv.isnull().sum()/surv.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [14]:
missing_data.head(3)

In [15]:
missing_data

### 03-02) 결측치 처리

In [16]:
surv = surv.drop(['comments'], axis=1)
surv = surv.drop(['state'], axis=1)
surv = surv.drop(['Timestamp'], axis=1)

surv.isnull().sum().max()
surv.head()

### 03-03) Default 값 정리해주기

In [17]:
D_Int = 0
D_Str = 'NaN'
D_Flo = 0.0

Int_F = ['Age']
Str_F = ['Gender','Country', 'self_employed', 'family_history', 'treatment', 
         'work_interfere', 'no_employees', 'remote_work', 'tech_company', 
         'anonymity', 'leave', 'mental_health_consequence', 'phys_health_consequence', 
         'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview',
         'mental_vs_physical', 'obs_consequence', 'benefits', 'care_options', 
         'wellness_program', 'seek_help']
Flo_F = []

for feature in surv:
    if feature in Int_F:
        surv[feature] = surv[feature].fillna(D_Int)
    elif feature in Str_F:
        surv[feature] = surv[feature].fillna(D_Str)
    elif feature in Flo_F:
        surv[feature] = surv[feature].fillna(D_Flo)
    else:
        print('Error: Feature %s not recognized.' % feature)
surv.head()   

In [18]:
surv = surv.drop(['self_employed'], axis=1)

In [19]:
surv.head()

In [20]:
gender = surv['Gender'].str.lower()

In [21]:
print(gender)

In [22]:
gender = surv['Gender'].unique()

In [23]:
print(gender)

In [24]:
M_str = ['M', 'Male', 'male', 'm', 'Male-ish', 'maile', 'Cis Male', 'Mal', 
         'Male (CIS)', 'Make', 'Male ', 'Man', 'msle', 'Mail', 'Malr', 'cis male', 
         'Cis Man' ]
T_str = ['Trans-female', 'something kinda male?', 'queer/she/they', 'non-binary',
         'Nah', 'All', 'Enby', 'fluid', 'Genderqueer', 'Androgyne', 'Agender', 
         'Guy (-ish) ^_^', 'male leaning androgynous', 'Trans woman', 
         'Neuter', 'Female (trans)', 'queer', 'ostensibly male, unsure what that really means']
F_str = ['Female', 'female', 'Cis Female', 'F', 'Woman', 'f', 'Femake', 'woman',
         'Female ', 'cis-female/femme', 'Female (cis)', 'femail' ]

for (row, col) in surv.iterrows():
    if str.lower(col.Gender) in M_str:
        surv['Gender'].replace(to_replace=col.Gender, value='male', inplace=True)
    if str.lower(col.Gender) in F_str:
        surv['Gender'].replace(to_replace=col.Gender, value='female', inplace=True)
    if str.lower(col.Gender) in T_str:
        surv['Gender'].replace(to_replace=col.Gender, value='trans', inplace=True)
        
stk_list = ['A little about you', 'p']
surv = surv[~surv['Gender'].isin(stk_list)]

print(surv['Gender'].unique())

In [25]:
surv.head()

### 04. 데이터 전처리

* LabelEncoding은 데이터 전처리를 할 때 자료 데이터를 숫자로 치환하여 머신러닝을 할 수 있게 해주는 sklearn의 preprocessing 방법이다.

In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
#complete missing age with mean
surv['Age'].fillna(surv['Age'].median(), inplace = True)

# Fill with media() values < 18 and > 120
s = pd.Series(surv['Age'])
s[s<18] = surv['Age'].median()
surv['Age'] = s
s = pd.Series(surv['Age'])
s[s>120] = surv['Age'].median()
surv['Age'] = s

#Ranges of Age
surv['age_range'] = pd.cut(surv['Age'], [0,20,30,65,100], labels=["0-20", "21-30", "31-65", "66-100"], include_lowest=True)

In [29]:
#There are only 0.20% of self work_interfere so let's change NaN to "Don't know
#Replace "NaN" string from defaultString

surv['work_interfere'] = surv['work_interfere'].replace([D_Str], 'Don\'t know' )
print(surv['work_interfere'].unique())

### 05. 데이터 인코딩

In [31]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.preprocessing import binarize, LabelEncoder, MinMaxScaler

In [32]:
#Encoding data
labelDict = {}
for feature in surv:
    le = preprocessing.LabelEncoder()
    le.fit(surv[feature])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    surv[feature] = le.transform(surv[feature])
    # Get labels
    labelKey = 'label_' + feature
    labelValue = [*le_name_mapping]
    labelDict[labelKey] =labelValue
    
for key, value in labelDict.items():     
    print(key, value)

#Get rid of 'Country'
surv = surv.drop(['Country'], axis= 1)
surv.head()

In [33]:
#missing data
total = surv.isnull().sum().sort_values(ascending=False)
percent = (surv.isnull().sum()/surv.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
print(missing_data)

### 06. 공분산행렬
* Covariance Matrix. Variability comparison between categories of variables

In [34]:
#correlation matrix
corrmat = surv.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);
plt.show()

In [35]:
#treatment correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'treatment')['treatment'].index
cm = np.corrcoef(surv[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

### 07. 시각화 (데이터 상관관계)

* 연령별 분포와 밀도

In [36]:
# Distribiution and density by Age
plt.figure(figsize=(12,8))
sns.distplot(surv["Age"], bins=24)
plt.title("Distribuition and density by Age")
plt.xlabel("Age")

* 치료력에 따른 분포 (치료력 유, 무)

In [37]:
# Separate by treatment or not

g = sns.FacetGrid(surv, col='treatment', size=5)
g = g.map(sns.distplot, "Age")

In [39]:
# Let see how many people has been treated
plt.figure(figsize=(12,8))
labels = labelDict['label_Gender']
g = sns.countplot(x="treatment", data=surv)
g.set_xticklabels(labels)

plt.title('Total Distribuition by treated or not')

* 0은 female, 1은 male 'label_Gender'에 따라 얼마나 치료를 받았는지 확인

In [40]:
o = labelDict['label_age_range']

g = sns.factorplot(x="age_range", y="treatment", hue="Gender", data=surv, kind="bar",  ci=None, size=5, aspect=2, legend_out = True)
g.set_xticklabels(o)

plt.title('Probability of mental health condition')
plt.ylabel('Probability x 100')
plt.xlabel('Age')
# replace legend labels

new_labels = labelDict['label_Gender']
for t, l in zip(g._legend.texts, new_labels): t.set_text(l)

# Positioning the legend
g.fig.subplots_adjust(top=0.9,right=0.8)

plt.show()